In [1]:
import numpy as np
from scipy import misc
import skimage
import skimage.color as color
import skimage.io as io
import matplotlib.pyplot as plt

In [2]:
# (Vsort, Dsort) = eigsort(V, D)
#
# Sorts a matrix eigenvectors and a matrix of eigenvalues in order
# of eigenvalue size, largest eigenvalue first and smallest eigenvalue
# last.
#
# Example usage:
# (D, V) = np.linalg.eig(A)
# (Vnew, Dnew) = eigsort(V, D)
#
# Tim Marks 2002
def eigsort(V, D):
    eigvals = D if D.ndim == 1 else np.diag(D)

    # Sort the eigenvalues from largest to smallest. Store the sorted
    # eigenvalues in the column vector eigvec.
    lohival = np.sort(eigvals, axis=0)
    lohiindex = np.argsort(eigvals, axis=0)
    eigvec = np.flipud(lohival)
    index = np.flipud(lohiindex)
    Dsort = np.diag(eigvec)

    # Sort eigenvectors to correspond to the ordered eigenvalues. Store sorted
    # eigenvectors as columns of the matrix vsort.
    M = len(eigvec)
    Vsort = np.zeros((M, M))
    for i in range(M):
        Vsort[:,i] = V[:,index[i]]

    return (Vsort, Dsort)

In [3]:
# Takes as input a 3xN ZERO-MEANED matrix, with each column a 3-component representation of the color of the pixel
# Outputs 3x3 sorted eigenvector matrix
def pca(Z):
    n = np.shape(Z)[1]
    cov = np.dot(Z, np.transpose(Z))/n
    (Dold, Vold) = np.linalg.eig(cov)
    (V, D) = eigsort(Vold, Dold)
    return V

In [4]:
#Converts RGB to HSI color scheme
#Input: 3-channel image
#Output: 3-channel image
def rgb_to_hsi(test_image):
    return color.rgb2hsv(test_image)

#Converts HSI back to RGB color scheme for imshow to work. Do not imshow from HSI scheme
#Input: 3-channel image
#Output: 3-channel image
def hsi_to_rgb(test_image):
    return color.hsv2rgb(test_image)

In [5]:
#Converts RGB to Lab color scheme
#Input: 3-channel image
#Output: 3-channel image
def rgb_to_lab(test_image):
    return color.rgb2lab(test_image)

#Converts Lab back to RGB color scheme for imshow to work. Do not imshow from HSI scheme
#Input: 3-channel image
#Output: 3-channel image
def lab_to_rgb(test_image):
    return color.lab2rgb(test_image)

In [6]:
def rgb_to_rgb(test_image):
    return test_image

In [7]:
# Takes as input a 3xN matrix, colour represenation (rep), the number of eigenvectors to use, and the name of the image
# rep:
# 'rgb', 'hsi', or 'lab'
def pca_reconstruction(rgb_im, rep, n, name, noise='normal'):
    
    rep_image = {
            'rgb': rgb_to_rgb(rgb_im),
            'hsi': rgb_to_hsi(rgb_im),
            'lab': rgb_to_lab(rgb_im),
        }[rep]
    
    X = np.transpose(rep_image.reshape(-1, rep_image.shape[-1]))
    mean = np.mean(X, axis=1)
    Z = np.subtract(X, np.vstack(mean))
    U = pca(Z)
    
    c = np.dot(np.transpose(U[:, 0:n]), Z)
    reconstructed = np.dot(U[:, 0:n], c) + np.vstack(mean)
    
    
    rec_image = np.transpose(reconstructed).reshape(rgb_im.shape)
    
    #Threshold out of boundary values
    if rep == 'rgb':
        rec_image[rec_image <= 0] = 0
        rec_image[rec_image >= 255] = 255
    elif rep == 'hsi':
        rec_image[rec_image <= 0] = 0
        rec_image[rec_image >= 1] = 1
        
    rgb_rec_image = {
            'rgb': np.uint8(rec_image) if noise == 'normal' else rgb_to_rgb(rec_image),
            'hsi': hsi_to_rgb(rec_image),
            'lab': lab_to_rgb(rec_image),
        }[rep]
    
    plt.imsave(name + rep + str(i) + ".png", rgb_rec_image)

In [ ]:
import os
images = [f for f in os.listdir("Images") if (f.endswith('.jpg') or f.endswith('.png'))]

for i in range(1,2):
    for im in images:
        img = misc.imread('Images/' + im)
        name = "Results/" + im.split('.')[0]
        
        pca_reconstruction(img, 'rgb', i, name)
        pca_reconstruction(img, 'hsi', i, name)
        pca_reconstruction(img, 'lab', i, name)